In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ln -s /content/drive/'My Drive'/Zalo_HT /content/

ln: failed to create symbolic link '/content/Zalo_HT': File exists


In [ ]:
cd Zalo_HT

/content/drive/My Drive/Zalo_HT


In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

HUGGING_FACE_USER_NAME = "namngduc"

peft_model_id = f"{HUGGING_FACE_USER_NAME}/GenerationText-Bloom"
config = PeftConfig.from_pretrained(peft_model_id)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [ ]:
config.base_model_name_or_path

'bigscience/bloom-1b1'

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

if device == "cuda":
    model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map="auto",
    )
elif device == "mps":
    model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        config.base_model_name_or_path, device_map={"": device}, low_cpu_mem_usage=True
    )



In [ ]:
# Load the LoRa model
model = PeftModel.from_pretrained(model, peft_model_id)

In [ ]:
model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1536)
        (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): Linear8bitLt(
                in_features=1536, out_features=4608, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4608, bias=False)
                )
                (lora_embedding_A): Parame

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. # noqa: E501
                  ### Instruction:
                  {instruction}
                  ### Input:
                  {input}
                  ### Response:
                """
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request. # noqa: E501
                ### Instruction:
                {instruction}
                ### Response:
                """

In [ ]:
from transformers import GenerationConfig

def make_infer(instruction,
        input=None,
        temperature=0.1,
        top_p=0.75,
        top_k=40,
        num_beams=4,
        max_new_tokens=128,
        **kwargs,
    ):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()


In [ ]:
instruction = "Describe the worst vacation you've ever had"
make_infer(instruction)

"The worst vacation I've ever had was the one I had when I was a teenager. I went on a camping trip with my friends and it was a great time, but it was also the worst vacation I've ever had. We had a great time, but it was also the worst vacation I have ever had. We had a great time, but it was also the worst vacation I have ever had. We had a great time, but it was also the worst vacation I have ever had. # noqa: E501 # noqa: E501 # noqa: E501 # noqa: E501 # noqa: E"